In [1]:
import ccxt
import pandas as pd
import numpy as np

In [2]:
# Initialize Binance USDT-M Futures exchange
exchange = ccxt.binanceusdm()

# List of symbols (Perpetual Futures contracts)
symbols = [
    "BTC/USDT",
    "ETH/USDT",
    "XRP/USDT",
    "BNB/USDT",
    "DOGE/USDT",
    "ADA/USDT",
    "SOL/USDT",
    "TRX/USDT",
    "AVAX/USDT",
    "LINK/USDT",
    "1000SHIB/USDT",
    "TON/USDT",
    "DOT/USDT",
    "SUI/USDT",
    "XLM/USDT",
    "HBAR/USDT",
    "BCH/USDT",
    "UNI/USDT",
    "1000PEPE/USDT",
    "LTC/USDT",
    "NEAR/USDT",
    "APT/USDT",
    "ICP/USDT",
    "AAVE/USDT",
    "RNDR/USDT",
    "FET/USDT",
    "ARB/USDT",
    "FIL/USDT",
    "ALGO/USDT",
    "FTM/USDT",
    "ATOM/USDT",
    "OP/USDT",
    "IMX/USDT",
    "INJ/USDT",
    "THETA/USDT",
    "SEI/USDT",
    "WLD/USDT",
    "1000FLOKI/USDT",
    "JASMY/USDT",
    "RUNE/USDT",
    "LDO/USDT",
    "GALA/USDT",
    "SAND/USDT",
    "MKR/USDT",
    "QNT/USDT",
    "EOS/USDT",
    "APE/USDT",
    "CRV/USDT",
    "DYDX/USDT",
    "ENS/USDT",
    "SUSHI/USDT",
    "TRB/USDT",
    "TURBO/USDT",
]

symbols

['BTC/USDT',
 'ETH/USDT',
 'XRP/USDT',
 'BNB/USDT',
 'DOGE/USDT',
 'ADA/USDT',
 'SOL/USDT',
 'TRX/USDT',
 'AVAX/USDT',
 'LINK/USDT',
 '1000SHIB/USDT',
 'TON/USDT',
 'DOT/USDT',
 'SUI/USDT',
 'XLM/USDT',
 'HBAR/USDT',
 'BCH/USDT',
 'UNI/USDT',
 '1000PEPE/USDT',
 'LTC/USDT',
 'NEAR/USDT',
 'APT/USDT',
 'ICP/USDT',
 'AAVE/USDT',
 'RNDR/USDT',
 'FET/USDT',
 'ARB/USDT',
 'FIL/USDT',
 'ALGO/USDT',
 'FTM/USDT',
 'ATOM/USDT',
 'OP/USDT',
 'IMX/USDT',
 'INJ/USDT',
 'THETA/USDT',
 'SEI/USDT',
 'WLD/USDT',
 '1000FLOKI/USDT',
 'JASMY/USDT',
 'RUNE/USDT',
 'LDO/USDT',
 'GALA/USDT',
 'SAND/USDT',
 'MKR/USDT',
 'QNT/USDT',
 'EOS/USDT',
 'APE/USDT',
 'CRV/USDT',
 'DYDX/USDT',
 'ENS/USDT',
 'SUSHI/USDT',
 'TRB/USDT',
 'TURBO/USDT']

In [3]:
def fetch_ohlcv(symbol, since, timeframe="1d"):
    """
    Fetch historical OHLCV data for a given symbol.
    """

    try:
        ohlcv = exchange.fetch_ohlcv(
            symbol, timeframe=timeframe, since=since, limit=1000
        )
        df = pd.DataFrame(
            ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
        )
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df.set_index("datetime", inplace=True)
        return df["close"]
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")
        return None

In [4]:
def filter_and_save_correlations(correlations, threshold, output_file):
    """
    Filter symbols based on correlation threshold and save to a text file.
    """
    # Filter symbols with correlation higher than the threshold
    filtered_symbols = [
        symbol for symbol, corr in correlations.items() if corr > threshold
    ]
    # Format symbols to match the required text format
    formatted_symbols = ",".join(
        [
            f"BINANCE:{symbol.replace('/', '').replace('1000', '1000').replace('1M', '1M')}.P"
            for symbol in filtered_symbols
        ]
    )
    # Save to text file
    with open(output_file, "w") as file:
        file.write(formatted_symbols)
    print(f"Filtered symbols saved to {output_file}")

In [5]:
# Fetch historical prices starting from January 1, 2023
price_data = {}
since = exchange.parse8601("2023-01-01T00:00:00Z")

for symbol in symbols:
    print(f"Fetching data for {symbol}")
    data = fetch_ohlcv(symbol, since)
    if data is not None:
        price_data[symbol] = data

Fetching data for BTC/USDT
Fetching data for ETH/USDT
Fetching data for XRP/USDT
Fetching data for BNB/USDT
Fetching data for DOGE/USDT
Fetching data for ADA/USDT
Fetching data for SOL/USDT
Fetching data for TRX/USDT
Fetching data for AVAX/USDT
Fetching data for LINK/USDT
Fetching data for 1000SHIB/USDT
Fetching data for TON/USDT
Fetching data for DOT/USDT
Fetching data for SUI/USDT
Fetching data for XLM/USDT
Fetching data for HBAR/USDT
Fetching data for BCH/USDT
Fetching data for UNI/USDT
Fetching data for 1000PEPE/USDT
Fetching data for LTC/USDT
Fetching data for NEAR/USDT
Fetching data for APT/USDT
Fetching data for ICP/USDT
Fetching data for AAVE/USDT
Fetching data for RNDR/USDT
Could not fetch data for RNDR/USDT: binanceusdm does not have market symbol RNDR/USDT
Fetching data for FET/USDT
Fetching data for ARB/USDT
Fetching data for FIL/USDT
Fetching data for ALGO/USDT
Fetching data for FTM/USDT
Fetching data for ATOM/USDT
Fetching data for OP/USDT
Fetching data for IMX/USDT
Fetch

In [6]:
# Create a DataFrame of prices and calculate daily returns
prices_df = pd.DataFrame(price_data)
prices_df = prices_df.dropna(axis=1, how="any")  # Remove symbols with insufficient data
returns_df = prices_df.pct_change().dropna()

returns_df

,BTC/USDT,ETH/USDT,XRP/USDT,BNB/USDT,DOGE/USDT,ADA/USDT,SOL/USDT,TRX/USDT,AVAX/USDT,LINK/USDT,...,SAND/USDT,MKR/USDT,QNT/USDT,EOS/USDT,APE/USDT,CRV/USDT,DYDX/USDT,ENS/USDT,SUSHI/USDT,TRB/USDT
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02,0.003353,0.011059,0.027770,0.002906,0.016382,0.014812,0.128496,0.007306,0.027808,0.011198,...,0.055126,-0.000976,0.027078,0.003413,0.022257,0.011385,0.037752,0.035337,0.019129,0.018977
2023-01-03,0.000072,0.000824,-0.012935,0.003672,-0.012474,-0.003550,0.183586,-0.006890,0.019262,-0.012480,...,0.019531,-0.015234,-0.010324,-0.002268,0.012152,-0.007505,-0.017766,-0.005851,0.005214,-0.016194
2023-01-04,0.010494,0.034722,0.010483,0.051104,0.037184,0.058987,0.005928,-0.000913,0.061088,0.033286,...,0.034722,0.027965,0.041822,0.030682,0.012756,0.047259,0.014643,0.046103,0.026971,0.037860
2023-01-05,-0.001087,-0.004226,-0.026801,-0.008123,-0.021894,0.005981,0.000970,-0.019371,-0.027253,-0.025495,...,-0.014811,0.016014,-0.010460,-0.017641,-0.011855,-0.021661,0.000000,-0.038616,-0.004040,0.000000
2023-01-06,0.007133,0.013956,0.018656,0.012674,0.012591,0.034560,0.004844,-0.022736,0.002470,0.009369,...,0.013625,0.025636,0.010029,0.004489,0.008248,0.007380,0.029711,0.023408,0.057809,-0.011102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13,0.013897,0.006463,0.036061,0.027684,0.007414,-0.001514,-0.009565,-0.020592,-0.006457,-0.004978,...,0.013329,0.038140,0.019740,0.009099,0.030013,-0.047699,-0.027039,0.056876,-0.009023,-0.001789
2024-12-14,0.000090,-0.009474,-0.009207,-0.014000,-0.026796,-0.054317,-0.021672,-0.028801,-0.055910,0.004623,...,-0.068185,-0.086545,-0.025950,-0.060415,-0.062378,0.003515,-0.037981,-0.028890,-0.049169,-0.055867
2024-12-15,0.030375,0.023238,0.019836,0.005534,0.021028,0.037159,0.019878,0.011763,0.018764,0.004808,...,0.027972,0.063336,0.011737,0.037428,0.050728,-0.012259,0.052961,0.137526,0.079914,0.025790


In [7]:
# Calculate correlation of each coin's returns with BTC
btc_returns = returns_df["BTC/USDT"]
correlations = {}
for symbol in returns_df.columns:
    if symbol != "BTC/USDT":
        corr = returns_df[symbol].corr(btc_returns)
        correlations[symbol] = corr
        
correlations

{'ETH/USDT': 0.8054726854317124,
 'XRP/USDT': 0.4084301247251129,
 'BNB/USDT': 0.6309225822947199,
 'DOGE/USDT': 0.7082482220478284,
 'ADA/USDT': 0.6543765592356756,
 'SOL/USDT': 0.6655952094674347,
 'TRX/USDT': 0.22769602565118885,
 'AVAX/USDT': 0.644526984470792,
 'LINK/USDT': 0.5762294230705297,
 '1000SHIB/USDT': 0.5847570625413959,
 'DOT/USDT': 0.6414479996708768,
 'XLM/USDT': 0.376626494928501,
 'HBAR/USDT': 0.4213736472728002,
 'BCH/USDT': 0.5578943722486451,
 'UNI/USDT': 0.5235090271299384,
 'LTC/USDT': 0.5681158581319208,
 'NEAR/USDT': 0.6135279857842968,
 'APT/USDT': 0.5408165370166192,
 'ICP/USDT': 0.5551207178952569,
 'AAVE/USDT': 0.5644198763629259,
 'FIL/USDT': 0.6171432704730714,
 'ALGO/USDT': 0.600336755019689,
 'FTM/USDT': 0.5980572183209727,
 'ATOM/USDT': 0.6178209529626025,
 'OP/USDT': 0.5306245895529038,
 'IMX/USDT': 0.5916086004889558,
 'INJ/USDT': 0.5754333965713385,
 'THETA/USDT': 0.6466126405500512,
 'JASMY/USDT': 0.5006206326106473,
 'RUNE/USDT': 0.6551980081408

In [8]:
# Sort correlations in descending order
sorted_correlations = dict(
    sorted(correlations.items(), key=lambda item: item[1], reverse=True)
)

# Print correlations
print("\nCorrelation of each coin's returns with BTC:")
for symbol, corr in sorted_correlations.items():
    print(f"{symbol}: {corr:.4f}")


Correlation of each coin's returns with BTC:
ETH/USDT: 0.8055
DOGE/USDT: 0.7082
SOL/USDT: 0.6656
RUNE/USDT: 0.6552
ADA/USDT: 0.6544
THETA/USDT: 0.6466
AVAX/USDT: 0.6445
DOT/USDT: 0.6414
BNB/USDT: 0.6309
ATOM/USDT: 0.6178
FIL/USDT: 0.6171
NEAR/USDT: 0.6135
ALGO/USDT: 0.6003
FTM/USDT: 0.5981
IMX/USDT: 0.5916
1000SHIB/USDT: 0.5848
EOS/USDT: 0.5829
LINK/USDT: 0.5762
INJ/USDT: 0.5754
LTC/USDT: 0.5681
AAVE/USDT: 0.5644
BCH/USDT: 0.5579
CRV/USDT: 0.5557
ICP/USDT: 0.5551
APT/USDT: 0.5408
GALA/USDT: 0.5406
SAND/USDT: 0.5374
OP/USDT: 0.5306
SUSHI/USDT: 0.5299
QNT/USDT: 0.5250
UNI/USDT: 0.5235
LDO/USDT: 0.5226
DYDX/USDT: 0.5218
APE/USDT: 0.5205
JASMY/USDT: 0.5006
ENS/USDT: 0.4720
MKR/USDT: 0.4617
HBAR/USDT: 0.4214
XRP/USDT: 0.4084
TRB/USDT: 0.3960
XLM/USDT: 0.3766
TRX/USDT: 0.2277


In [9]:
# Set correlation threshold and output file name
threshold = 0.6
output_file = "HighCorrelation.txt"

# Filter and save symbols with high correlation
filter_and_save_correlations(sorted_correlations, threshold, output_file)

Filtered symbols saved to HighCorrelation.txt
